# Color Block Portraits
#### The overall goal of this project is to take a given photo and create a color block version of it.

## Import the Required Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
import cv2
import ntpath
import os

## Create the ColorBlock Class
#### This class processes a given image using its contours to create the output color block image.

In [4]:
class ColorBlock():
    def __init__(self, image_path, color_count):
        self.file = ntpath.basename(image_path).split(".")[0]
        self.source = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        self.color_count = color_count

    def blur_image(self):
        cv2_base_dir = os.path.dirname(os.path.abspath(cv2.__file__))
        haar_model = os.path.join(cv2_base_dir, 'data/haarcascade_frontalface_default.xml')
        face_detect = cv2.CascadeClassifier(haar_model)
        face_data = face_detect.detectMultiScale(self.source, 1.01, 5)
        
        for (x, y, w, h) in face_data:
            cv2.rectangle(self.source, (x, y), (x + w, y + h), (0, 255, 0), 2)
            roi = self.source[y:y+h, x:x+w]
            roi = cv2.GaussianBlur(roi, (133, 133), 50)
            self.source[y:y+roi.shape[0], x:x+roi.shape[1]] = roi

    def resize_image(self):
        """
        This function resizes the given image to match a preset picture ratio
        """
        (h, w) = self.source.shape[:2]
        if h > w:
            dim = (int(w * 2000 / float(h)), 2000)
        else:
            dim = (2000, int(h * 2000 / float(w)))
        return cv2.resize(self.source, dim, interpolation=cv2.INTER_AREA)

    def clean_image(self, image):
        """
        This function takes a given image and does denoising and Morphomat operations to clean it
        """
        cleaned_pic = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8, 8))
        cleaned_pic = cv2.morphologyEx(cleaned_pic, cv2.MORPH_OPEN, kernel, cv2.BORDER_REPLICATE)
        cleaned_pic = cv2.morphologyEx(cleaned_pic, cv2.MORPH_CLOSE, kernel, cv2.BORDER_REPLICATE)
        return cleaned_pic

    def kmeans_image(self, image):
        """
        This function takes a given image and returns its K-mean version
        """
        w, h, dim = tuple(image.shape)
        image_array = np.reshape(image, (w * h, dim))
        image_array_sample = shuffle(image_array, random_state=0)[:1000]
        kmeans = KMeans(n_clusters=self.color_count, random_state=42).fit(image_array_sample)
        labels = kmeans.predict(image_array)
        new_img = self.recreate_image(kmeans.cluster_centers_, labels, w, h)
        return new_img, kmeans.cluster_centers_

    def recreate_image(self, codebook, labels, width, height):
        """
        This function recreates an image using stored information regarding image size, colors, and labels
        """
        vfunc = lambda x: codebook[labels[x]]
        out = vfunc(np.arange(width*height))
        return np.resize(out, (width, height, codebook.shape[1]))

    def generate_image(self):
        """
        This function calls all the helper functions define earlier (above) to run our algorithm to generate
        the color block image
        """
        self.blur_image()
        img_source = self.resize_image()
        cleaned_img = self.clean_image(img_source)
        cleaned_img = np.array(cleaned_img, dtype="uint8")/255
        kmeans_image, _ = self.kmeans_image(cleaned_img)

        cv2.imwrite(f"./outputs/{self.file}-output.png", cv2.cvtColor(kmeans_image.astype('float32')*255, cv2.COLOR_BGR2RGB))

## Run the Project
#### The following code runs our entire project from start to finish.

In [5]:
# portrait = ColorBlock("inputs/person3.PNG", 15)
# portrait = ColorBlock("inputs/person.png", 15)
portrait = ColorBlock("inputs/person5.jpg", 15)
portrait.generate_image()